In [17]:
# Se importan las libreriasa necesraias parael desarrollo del proyecto
import pandas as pd
import numpy as np
import re
import requests
import json
from bs4 import BeautifulSoup
import whois
import builtwith

In [18]:
# Función para eliminar tildes de las vocales
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [19]:
# URL del sitio inicial del grupo de investigación donse se va realizar el Web Scraping
url= ('https://www.agrosavia.co/nosotros/grupos-de-investigacion')

In [20]:
# se modifican cabeceras HTTP, incluyendo User-Agent
user agent
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache",
"dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
}
r = requests.get(url, headers=headers)

In [21]:
#Se obtiene e imprime información sobre el propietario del sitio
print(whois.whois(url))

{
  "domain_name": "agrosavia.co",
  "registrar": "Central Comercializadora de Internet S.A.S",
  "whois_server": null,
  "referral_url": null,
  "updated_date": "2018-05-17 14:07:16",
  "creation_date": "2018-04-12 17:39:16",
  "expiration_date": "2023-04-12 17:39:16",
  "name_servers": [
    "ns1.corpoica.org.co",
    "ns2.corpoica.org.co"
  ],
  "status": "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
  "emails": null,
  "dnssec": "unsigned",
  "name": "REDACTED FOR PRIVACY",
  "org": "Privacy Protect, LLC (PrivacyProtect.org)",
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "MA",
  "registrant_postal_code": "REDACTED FOR PRIVACY",
  "country": "US"
}


In [ ]:
# Se obtiene la tecnología utilizada en el diseño del sitio web, la cual condicionará el tipo de web scraping aplicado
builtwith.builtwith(url)

{'font-scripts': ['Google Font API'],
 'maps': ['Google Maps'],
 'web-frameworks': ['Twitter Bootstrap'],
 'javascript-frameworks': ['jQuery']}

In [33]:
# La función request obtiene el contenido bruto de la página web
response = requests.get(url)
indice = re.compile(r'\d{10}$')
# obtener la estructura anidada de la pagina a través de la función BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
# Se definen las columnas que tendrá el dataframe
df = pd.DataFrame(columns=['titulo_grupo','url_grupo','lider_grupo','categoria_grupo','correo_grupo','anocrea_grupo','areaconoce_grupo','prognal_grupo','prognalsec_grupo','lineas_grupo'])
count=0
countlis=0
# se realiza un recorrido por todos los elementos a (enlaces) que tiene la página principal de grupos de investigación
for link in soup.find_all('a'):
    count+=1
    # Se crea la condición para determinar si el hipervinculo corresponde a un enlace a un grupo específico
    if 'https://www.agrosavia.co/nosotros/grupos-de-investigacion/' in link.get('href'):
         #Se establace el valor de la columa url_grupo con el hipervinculo activo
         url_grupo = link.get('href')
         response2 =  requests.get(url_grupo)
         # Se obtiene la estructura anidad del grupo especifico y se asignan los valores a las columnas
         soup2 = BeautifulSoup(response2.content, 'html.parser')   
         titulo_grupo = soup2.main.header.h1.string
         a_tag = soup2.find(class_="details") 
         lider_grupo = a_tag.a.string 
         # Se realiza análisis horizontal para encontrar los hermanos del elemento y poder asignar el valor al campo categoria_grupo
         i_tag = soup2.find(class_="icon-teamwork")
         i_tag2 = i_tag.next_sibling
         categoria_grupo = (str(i_tag2.next_sibling)).strip()
         # Se realiza análisis horizontal para encontrar los hermanos del elemento tag y poder asignar el valor al campo categoria_grupo
         a_tag2 = soup2.find(class_="entry-text")
         a_tag3 = a_tag2.find_all('a')
         if len(a_tag3) > 1:
            correo_grupo = a_tag3[1].string
         else:
             correo_grupo = 'null'
         # Recorre todos los elementos strong del tag para luego a través de análisis horizontal encontrar los hermanos y asignar valor a las columnas 
         for link2 in a_tag2.find_all('strong'):  
            if link2.string == 'Año de Creación':
                anocrea_grupo = (str(link2.next_sibling)).strip()
                anocrea_grupo = anocrea_grupo.replace(':','')
            if link2.string == 'Área de conocimiento':
                areaconoce_grupo = link2.next_sibling
                areaconoce_grupo = areaconoce_grupo.replace(':','')
            if link2.string == 'Programa nacional de ciencia y tecnología':
                prognal_grupo = link2.next_sibling
                prognal_grupo = prognal_grupo.replace(':','')
            if link2.string == 'Programa nacional de ciencia y tecnología (secundario)':
                prognalsec_grupo = link2.next_sibling
                prognalsec_grupo = prognalsec_grupo.replace(':','')
         # Recorre todos los elementos h3 del tag para luego a través de análisis horizontal encontrar los hermanos y asignar valor a la columna lineas_grupo   
         for link3 in a_tag2.find_all('h3'):         
            if  'LINEA' in normalize(str(link3).upper()): 
                target_td = link3.find_next_siblings('ol')
                lista = (target_td[0].contents)
                lineas_grupo = str(lista).strip()
                # Se limpia el strin resultante
                lineas_grupo = lineas_grupo.replace("<li>", "")
                lineas_grupo = lineas_grupo.replace("</li>","")
                lineas_grupo = lineas_grupo.replace('\\n',"")
                lineas_grupo = lineas_grupo.replace('[',"")
                lineas_grupo = lineas_grupo.replace(']',"")
                lineas_grupo = lineas_grupo.replace("''","")
                lineas_grupo = lineas_grupo.replace(" , ","")
                lineas_grupo = lineas_grupo.replace("<span>","")
                lineas_grupo = lineas_grupo.replace("</span>","")
         # Se llena el dataframe con los valores obtenidos por columna
         df.loc[count] = [ titulo_grupo, url_grupo, lider_grupo, categoria_grupo, correo_grupo, anocrea_grupo, areaconoce_grupo, prognal_grupo, prognalsec_grupo, lineas_grupo ]
          

In [32]:
# Se imprime el dataframe final
df

,titulo_grupo,url_grupo,lider_grupo,categoria_grupo,correo_grupo,anocrea_grupo,areaconoce_grupo,prognal_grupo,prognalsec_grupo,lineas_grupo
32,Ecología y agricultura inteligente para la sos...,https://www.agrosavia.co/nosotros/grupos-de-in...,Cesar Augusto Teran Chaves,A Convocatoria 894 de 2021 - con vigencia has...,cteran@agrosavia.co,1995,"Ciencias Agrícolas - Agricultura, Silvicultur...","Ciencia, Tecnología e Innovación en Ciencias ...","Ciencia, Tecnología e innovación en Ambiente,...",", Agricultura de precisión,Agroclimatología,Ca..."
33,Bioproductos y Bioprocesos Agropecuarios,https://www.agrosavia.co/nosotros/grupos-de-in...,Eddy Johana Bautista Bautista,B convocatoria 894 de 2021 - vigencia hasta l...,null,2017,"Ciencias Agrícolas - Agricultura, Silvicultur...",Biotecnología,"Ciencia, Tecnología e innovación en Ambiente,...",", Biología de sistemas,Diseño y selección de e..."
34,Bioprospección de Biomoléculas y Microorganism...,https://www.agrosavia.co/nosotros/grupos-de-in...,Carol Viviana Amaya Gómez,B Convocatoria 894 de 2021 - con vigencia hast...,camaya@agrosavia.co,2017,"Ciencias Agrícolas - Agricultura, Silvicultur...",Biotecnología,"Ciencia, Tecnología e innovación en Ambiente,...",", Biología de sistemas,Diseño y selección de e..."
35,Control biológico de plagas agrícolas,https://www.agrosavia.co/nosotros/grupos-de-in...,Gloria Patricia Barrera Cubillos,A1 Convocatoria 894 de 2021 - con vigencia has...,gbarrera@agrosavia.co,1994,"Ciencias Agrícolas - Agricultura, Silvicultur...",Biotecnología,"Ciencia, Tecnología e Innovación en Ciencias ...",", Biología de sistemas,Diseño y selección de e..."
36,Estudios Socioeconómicos - Corpoica,https://www.agrosavia.co/nosotros/grupos-de-in...,Antonio Maria Martínez Reina,C Convocatoria 894 de 2021 - con vigencia hast...,amartinezr@agrosavia.co,1998,"Ciencias Agrícolas - Agricultura, Silvicultur...","Ciencia, Tecnología e Innovación en Ciencias ...","Ciencia, Tecnología e Innovación en Ciencias ...",", Biología de sistemas,Diseño y selección de e..."
37,Grupo de investigación e innovación en salud y...,https://www.agrosavia.co/nosotros/grupos-de-in...,Byron Abdel Hernández Ortíz,A Convocatoria 894 de 2021 - con vigencia hast...,Enlace al Gruplac,1999,Ciencias Agrícolas - Ciencias Veterinarias - ...,"Ciencia, Tecnología e Innovación en Ciencias ...","Ciencia, Tecnología e Innovación en Salud",", Bienestar animal,Calidad e inocuidad de prod..."
38,Grupo de Análisis Espacio-Temporal y Manejo de...,https://www.agrosavia.co/nosotros/grupos-de-in...,Nancy del Carmen Barreto Triana,A según Convocatoria 894 de 2021 con vigencia ...,: nbarreto@agrosavia.co,2001,Ciencias Agrícolas - Ciencias Veterinarias - ...,"Ciencia, Tecnología e Innovación en Ciencias ...","Ciencia, Tecnología e Innovación en Salud",", Bienestar animal,Calidad e inocuidad de prod..."
39,Investigación y Vigilancia Integrada de la Res...,https://www.agrosavia.co/nosotros/grupos-de-in...,Maria del Pilar Donado Godoy,A Convocatoria 894 de 2021 - con vigencia hast...,Enlace al Gruplac,2007,Ciencias Naturales - Ciencias Biológicas - Bi...,"Ciencia, Tecnología e Innovación en Salud","Ciencia, Tecnología e Innovación en Ciencias A...",", Línea de investigación recolección de especí..."
40,Investigación en frutas del trópico,https://www.agrosavia.co/nosotros/grupos-de-in...,Demian Takumasa Kondo Rodríguez,A1 Convocatoria 894 de 2021 - con vigencia has...,Enlace al Gruplac,2000,"Ciencias Agrícolas - Agricultura, Silvicultur...","Ciencia, Tecnología e Innovación en Ciencias ...","Ciencia, Tecnología e Innovación en Ingeniería",", Línea de investigación recolección de especí..."
41,"Mejoramiento Genético Vegetal, Uso y Aprovecha...",https://www.agrosavia.co/nosotros/grupos-de-in...,Ivania Cerón Souza,A1 Convocatoria 894 de 2021 - con vigencia has...,Enlace al Gruplac,2003,Ciencias Naturales - Ciencias Biológicas - Ge...,"Ciencia, Tecnología e innovación en Ambiente,...","Ciencia, Tecnología e Innovación en Ciencias ...",

In [35]:
# Se realiza la conversión del dataframe al archivo .csv
df.to_csv('C:/Users/ovasquez/Downloads/Dataset_Final.csv')